# Henry PI 2: Machine Learning

Omarly Zerpa

# EDA (Analisis Exploratorio de Datos) 

Se extraen las librerias necesarias

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math


A continuación, se importa el archivo tipo parquet con los datos de entrenamiento a utilizar en un Pandas DataFrame.

In [ ]:
parquet= r'train.parquet'
df= pd.read_parquet(parquet, engine='auto')
df.head(6)

Obtenemos informacion basica del dataframe

In [ ]:
df.info()

In [ ]:
df.shape

Buscamos registros duplicados

In [ ]:
df.duplicated().value_counts()

Buscamos si existen valores nulos

In [ ]:
df.isnull().sum()

Verificaremos la columna long (longitud) y lat (latitud)

In [ ]:
#Buscamos los valores unicos
df['lat'].unique()

In [ ]:
df['long'].unique()

Creamos un nuevo dataframe para examinar mas de cerca si los datos de  lat y la long pertenecen a la region y estado indicado

In [3]:
df_latitud_longitud= df[['region','state', 'lat', 'long']]

Revisamos que que regiones tienen laitud y logitud nulos

In [ ]:
df_latitud_longitud[(df_latitud_longitud['lat'].isnull())]

Examinanos los valores de latitud y longitud de una region en especifico

In [ ]:
df_latitud_longitud[(df_latitud_longitud['region']=='denver')]

Se evidencia que cada region cuenta con una latitud y longitud especifica imposibilitando el cambio a un valor unico, ya que no se brindaria la ubicacion exacta del inmueble. Ademas desconozco el criterio empleado con dichos valores.

Ahora, se calcula el porcentaje de nulos en la columna latitud y longitud (es la misma cantidad) para evidenciar el impacto que tiene sobre el dataset

In [ ]:
nulos= df['lat'].isnull().sum()
total= len(df['lat'])
porcentaje= ((nulos/ total)*100)
print('El porcentaje de nulos en la columna lat', porcentaje)# es irrelevante para hacer una eliminacion de las columnas

# Transformaciones 

Se renombran las columnas lat y long

In [3]:
df.rename(columns={'lat':'latitude', 'long':'longitude'}, inplace= True)

Se confirma el cambio

In [ ]:
df.head(2)

Graficamos con dispersion la latitude y longitude, para confirmar que se encuentran en el pais indicado(EEUU) por lo general es: Estándar decimal simple	latitud:37.09024, longitud:-95.712891

In [ ]:
df.plot(kind='scatter', x='longitude', y='latitude', alpha=0.4, figsize=(10,7))

Existen puntos que se encuentran afueras de los limites de los EEUU, por esta razon no se tomara en cuenta estas columnas para el analisis

Analicemos las columnas laundry_options y parking_options 

In [ ]:
nulos= df['laundry_options'].isnull().sum()
total= len(df['laundry_options'])
porcentaje= ((nulos/ total)*100)
print('El porcentaje de nulos en la columna laundry_options', porcentaje)

In [ ]:
nulos= df['parking_options'].isnull().sum()
total= len(df['parking_options'])
porcentaje= ((nulos/ total)*100)
print('El porcentaje de nulos en la columna parking_options', porcentaje)

Como observamos representa un valor mayor al 10% de valores faltantes, podrian ser datos importantes como caracteristica del inmueble si no tuvise valores perdidos o faltantes, ahora para el estudio de este caso se obviaran estas columnas. 

In [ ]:
#Se eliminan las columnas mencionadas
df.drop(['laundry_options','parking_options'], axis= 1,  inplace=True)
df.shape

Los atribulos no seleccionados para este analisis son:

In [6]:
df.drop(['id', 'url', 'region_url', 'image_url', 'latitude', 'longitude'], axis= 1,  inplace=True)

Se confirman los cambios realizados

In [ ]:
df.info()

Se observa la informacion de la columna type, para saber si requiere algun cambio

In [ ]:
df['type'].unique()

In [ ]:
sns.catplot(y = "type", data = df, kind = "count")

Ahora, se crea la columna category_price utilizando la columna price

In [ ]:
valores= [-math.inf, 999, 1999,math.inf]
category= ['low','medium','high']
df2= df.assign(category_price= pd.cut(x=df['price'], bins=valores, labels=category, include_lowest=True))
df2

Se grafica los datos de la columna creada para visualizar la cantidad de inmubles que existe con price low, medium y high

In [ ]:
sns.countplot(x='category_price', data=df2) #equivelante a sns.countplot(df.Conversion2)
plt.show()

Se crea una columna target la cual me indique 1 para los inmuebles con price low y 0 para los inmuebles con price medium y high

In [ ]:
df2['target'] = df2['category_price'].apply(lambda x : 1 if x=='low' else 0)
df2

Se verifican los cambios realizados

In [ ]:
df2.info()

Correlaciones de variables

In [10]:
#se crea una lista con los atributos de los inmuebles
atributos= df2[['sqfeet','beds','baths', 'cats_allowed', 'dogs_allowed', 'smoking_allowed','wheelchair_access', 'electric_vehicle_charge','comes_furnished','target', 'price']]

In [ ]:
def plot_corre_heatmap(corr):
    '''
    Definimos una función para ayudarnos a graficar un heatmap de correlación
    '''
    plt.figure(figsize=(12,12))
    sns.heatmap(corr, cbar = True,  square = False, annot=True, fmt= '.2f'
                ,annot_kws={'size': 15},cmap= 'coolwarm')
    plt.xticks(rotation = 45)
    plt.yticks(rotation = 45)
    # Arreglamos un pequeño problema de visualización
    b, t = plt.ylim() # descubre los valores para bottom y top
    b += 0.5 # Agrega 0.5 al bottom
    t -= 0.5 # Resta 0.5 de la parte de top
    plt.ylim(b, t) # actualizar los valores  ylim(bottom, top)
    plt.show()

corr = atributos.corr()
plot_corre_heatmap(corr)

Defino mi tabla train a utilizar para el modelo

In [ ]:
train= df2 
train.head(2)

Dado la cantidad de nulos y valores no pertenecientes las columnas latitude y longitude,y escasa correlaciones entre las caracteristicas y el precio del inmueble, tomé la descripción para obtener palabras reelevantes que me permitan deducir si un inmueble tiene precio bajo o alto.

Vemos si existen descripciones faltantes en la columna descripción

In [ ]:
train['description'].isnull().sum()

Eliminamos las descripciones vacías porque mejora la performance del modelo

In [15]:
train.dropna(subset='description', inplace=True)

Revisamos que no hayan quedado registros con descripciones vacías

In [ ]:
train['description'].isnull().sum()

# Preprocesamiento de datos utilizando la columna descripcion y target

Importamos los modulos a utilizar

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

Obtenemos los stop words de esta biblioteca

In [18]:
sw = nltk.corpus.stopwords.words('spanish')

Nos quedamos con la lista de los textos de las descripciones

In [19]:
corpus = train['description']

Lo convertimos a lista

In [20]:
corpus = list(corpus)

Instanciamos el vectorizador con los stop words y un máximo de features de 1000

In [22]:
vectorizer = TfidfVectorizer(stop_words=sw, max_features=1000)

Vectorizamos los datos

In [23]:
x = vectorizer.fit_transform(corpus)

Vemos la cantidad de datos obtenidos

In [ ]:
x.shape

Obtenemos la variable objetivo

In [25]:
y = train['target']

Dimension de la variable objetivo

In [ ]:
y.shape

# Pipeline utilizando la columna descripcion y target

Importamos las librerias necesarias

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import recall_score, precision_score, accuracy_score
from sklearn.pipeline import Pipeline
import joblib

In [29]:
pipe_rf = Pipeline([('rf', RandomForestClassifier(n_estimators=50, random_state=0)),])

In [30]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
pipe_rf.fit(x_train, y_train)

Guardamos el Pipeline en un archivo Pickle

In [ ]:
joblib.dump(pipe_rf, 'modelo_random_forest2.pkl', compress=1)

Cargamos el archivo creado de nuestro modelo

In [33]:
rf = joblib.load('modelo_random_forest2.pkl')

Ajustamos el umbral de puntuacion (Adjust Score Threshold): Ajustar este valor cambia el nivel de confianza que debe tener el modelo en una predicción antes de considerar que la predicción es positiva. También cambia la cantidad de falsos negativos y falsos positivos que está dispuesto a tolerar en sus predicciones.

In [34]:
y_pred = np.where(rf.predict_proba(x_test) [:,1] > 0.2, 1, 0)

Generamos una matriz de confusion

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=rf.classes_)
disp = ConfusionMatrixDisplay(cm, display_labels=rf.classes_)
disp.plot()

Determinados la exhaustividad(recall), la precision y la exactitud(accuracy)

In [ ]:
rs = recall_score(y_test, y_pred)
ps = precision_score(y_test, y_pred)
ac = accuracy_score(y_test, y_pred)

print(f'Recall: {rs}')
print(f'Precision: {ps}')
print(f'Accuracy: {ac}')

# Traemos el archivo test

Cargamos el DataFrame

In [ ]:
parquet= r'test.parquet'
df_t= pd.read_parquet(parquet, engine='auto')
df_t.head(2)

In [ ]:
df_t.info()

Realizamos las mismas transformaciones que realizamos para el train

In [45]:
#renombramos las columnas
df_t.rename(columns={'lat':'latitude', 'long':'longitude'}, inplace=True)

In [46]:
#Eliminamos los columanas por las razones explicadas con anterioridad
df_t.drop(['id', 'url', 'region_url', 'laundry_options', 'parking_options', 'image_url','latitude', 'longitude'], axis=1, inplace= True)

In [47]:
#Creamos la columna category_price
valores2= [-math.inf, 999, 1999,math.inf]
category2= ['low','medium','high']
df_t2= df.assign(category_price= pd.cut(x=df['price'], bins=valores, labels=category, include_lowest=True))

In [48]:
#Creamos la columna target
df_t2['target']= df_t2['category_price'].apply(lambda x:1 if x=='low' else 0)

In [ ]:
#Se define la tabla test ha utilizar
test= df_t2
test.head(4)

Preparamos la columna description para utilizarla 

In [ ]:
# Vemos descripciones faltantes en la columna descripción
test['description'].isnull().sum()

In [ ]:
# Obtenemos los índices de los registros con descripciones vacías
sin_desc = test[test.description.isnull()].index
sin_desc

Se crea una funcion para para rellenar la descripciones faltantes en la columna description

In [54]:
def crearDescription(df, num_loc):
    '''
    Arma la descripción faltante con datos existentes

    df: DataFrame a analizar
    num_loc: índice del registro a cambiar
    '''
    cadena = ''
    # Evaluamos con que características contamos para armar
    # las descripciónes faltantes de las propiedades
    if not df.isnull().type[num_loc]: 
            cadena += df.type[num_loc] + ' '
    if not df.isnull().region[num_loc]:
        cadena += df.region[num_loc] + ' '
    if not df.isnull().beds[num_loc]:
        cadena += str(int(df.beds[num_loc])) + ' habitaciones '
    if not df.isnull().baths[num_loc]:
        cadena += str(int(df.baths[num_loc])) + ' baños'
    
    return cadena

Ahora, recorremos los índices de los registros con descripciones faltantes para completarlas

In [55]:
for idx in sin_desc:
    test.loc[idx,('description')] = crearDescription(test, idx) 

In [ ]:
# Verificamos que todas las propiedades tengan una descripción
test['description'].isnull().sum()

In [57]:
# Obtenemos los stop words de esta biblioteca 
sw = nltk.corpus.stopwords.words('spanish')

In [58]:
# Nos quedamos con la lista de los textos de las descripciones
corpus_t = test['description']

In [59]:
# Lo convertimos a lista
corpus_t = list(corpus_t)

In [ ]:
# Verificamos el tamaño de los datos obtendos, para observar que coincide 
# con la cantidad de registros a predecir
len(corpus_t)

Se realiza la modelizacion 

In [61]:
# Instanciamos el vectorizador con los stop words y un máximo de features de 1000
vectorizer_t = TfidfVectorizer(stop_words=sw, max_features=1000)

In [62]:
# Vectorizamos los datos
x_t = vectorizer_t.fit_transform(corpus_t)

In [ ]:
#Observamos la cantidad de datos obtenidos
x_t.shape

In [64]:
#Ajustamos el umbral de puntuacion (Adjust Score Threshold)
y_t_predict = np.where(rf.predict_proba(x_t) [:,1] > 0.2, 1, 0)

# Generación del CSV

Creamos el DataFrame que contendrá las predicciones obtenidas

In [65]:
datos = pd.DataFrame()

Creamos la columna pred con los datos obtendiso de la predicción

In [66]:
datos['pred'] = y_t_predict

Lo guardamos con formato CSV

In [67]:
datos.to_csv('OmarlyZerpa.csv', index=False)